In [8]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import re
from utils import load_model_and_files, get_model_details, reconstruct_word, merge_results, preprocess_text, preprocess_stopwords, preprocess_lemmatization, traduci_output, process_text_with_models, process_emotions_and_translate

In [9]:
# 1. Recupera i dettagli dei modelli
model_details = get_model_details()

# 2. Assegna manualmente le tuple a  separate
(model_1_id, model_1_type, model_1_files), (model_2_id, model_2_type, model_2_files), (model_3_id, model_3_type, model_3_files), (model_4_id, model_4_type, model_4_files) = model_details

# 3. Usa i dettagli per caricare i modelli
print(f"Caricamento modello 1: {model_1_id}")
model_1_pipeline = load_model_and_files(model_1_id, model_type=model_1_type)

print(f"Caricamento modello 2: {model_2_id}")
model_2_pipeline = load_model_and_files(model_2_id, model_type=model_2_type)

print(f"Caricamento modello 3: {model_3_id}")
model_3_pipeline = load_model_and_files(model_3_id, model_type=model_3_type)

print(f"Caricamento modello 4: {model_4_id}")
model_4_pipeline = load_model_and_files(model_4_id, model_type=model_4_type)


Caricamento modello 1: osiria/bert-italian-uncased-ner


Device set to use cpu


Caricamento modello 2: IVN-RIN/MedPsyNIT


Device set to use cpu


Caricamento modello 3: SamLowe/roberta-base-go_emotions


Device set to use cpu


Caricamento modello 4: Helsinki-NLP/opus-mt-it-en


In [10]:
def recupera_autori(autori_collection):
    # Recupera la lista di autori dal database
    autori_cursor = autori_collection.find()
    autori = []
    
    for autore in autori_cursor:
        nome_originale = autore["author"]  # Supponendo che il campo del nome dell'autore sia "author"
        autori.append(nome_originale)  # Aggiungi il nome originale
        
        # Genera una versione pulita del nome senza caratteri speciali e numeri
        nome_pulito = re.sub(r'[^a-zA-Z]', '', nome_originale)
        if nome_pulito:  # Evita di aggiungere stringhe vuote
            autori.append(nome_pulito)
    
    return autori

# Funzione per rimuovere emoji o stringhe delimitate da ':'
def rimuovi_emoji(messaggio):
    # Rimuove le sottostringhe racchiuse tra due punti, ad esempio :smiling_face:
    return re.sub(r':\w+?:', '', messaggio)

def analisi_semantica(text, autori):
    # Supponendo che i risultati siano già ottenuti
    risultato_ner = process_text_with_models(text, model_1_pipeline, model_2_pipeline, model_1_id, model_2_id)
    risultato_sentiment = process_emotions_and_translate(text, model_4_pipeline, model_3_pipeline)[0]

    # Filtrare i sentimenti per raggiungere uno score cumulativo di almeno 0.95
    selected_sentiments = []

    for sentiment in risultato_sentiment:
        if sentiment["score"] > 0.15:
            selected_sentiments.append(sentiment)
        

    # Controlla se la lista non è vuota
    if risultato_ner:
        # Itera sulla lista e converte i punteggi 'score' in float64
        for item in risultato_ner:
            item["score"] = float(item["score"])  # Converte np.float32 in float64
            # Se l'entità è un autore, etichettala come "PER"
            if item["word"] in autori:
                item["entity"] = "PER"
    else:
        print("La lista 'ner' è vuota!")

    risultato_sentiment = {item['label']: item['score'] for item in risultato_sentiment}

    selected_sentiments = {item['label']: item['score'] for item in selected_sentiments}

    # Creazione del dizionario
    data_semantic_analysis = {
        "ner": risultato_ner,
        "sentiment_analysis_relevant": selected_sentiments,
        "sentiment_analysis_full": risultato_sentiment
    }
    return data_semantic_analysis

In [11]:

# Connetti al database MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["performance_modelli"]
collection = db["post"]
autori_collection = db["autori"] 



autori = recupera_autori(autori_collection)

DIMENSIONE_BLOCCO = 50


cursor = collection.find().batch_size(DIMENSIONE_BLOCCO)
for documento in cursor:
    try:
        messaggio = documento.get("message")
        if not messaggio:
            continue  # Salta documenti senza messaggio
        # Pulisci il messaggio da emoji
        messaggio_pulito = rimuovi_emoji(messaggio)
        print(messaggio_pulito)
        
        # Esegui la pipeline
        risultato = analisi_semantica(messaggio_pulito,autori)
        
        # Aggiorna il documento con i risultati
        collection.update_one(
            {"_id": documento["_id"]},  # Filtro per identificare il documento
            {"$set": risultato}         # Aggiungi i risultati come nuove chiavi
        )

    except Exception as e:
        print(f"Errore nel processare il documento {documento['_id']}: {e}")

print("Elaborazione completata!")

Buonasera a tutte, finalmente trovo il coraggio di scrivere su questo gruppo nella speranza di trovare un po' di sostegno e conforto. Due mesi fa io e mio marito abbiamo scoperto di non poter concepire naturalmente. La prima reazione è stata di rabbia furiosa e di sconforto ed anche adesso, ogni volta che ci penso, ho la tentazione di rinchiudermi in una stanza buia e piangere. Il percorso di PMA mi terrorizza ma soprattutto mi devasta l'idea di dover affrontare ancora una difficoltà di vita dopo averne superate tante. Un bimbo sarebbe dovuto arrivare naturalmente, senza pensieri, una circostanza felice; e invece ora ho incubi su medici, iniezioni e dolori. Non riesco a superare lo shock e nessuno, a partire da mio marito, capisce cosa provo; nessuno si rende conto del fatto che l'intero peso fisico del percorso di PMA dovrò sopportarlo io. Poterne parlare qui è un sollievo, vi ringrazio
Ciao Ember, benvenuta  mi riconosco molto in quello che dici, e penso che tutte noi almeno in parte

In [22]:
# Connessione a MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Cambia con il tuo URI MongoDB
db = client["performance_modelli"]  # Sostituisci con il nome del tuo database
collezione = db["post"]  # Sostituisci con il nome della tua collezione

# Query per selezionare documenti
query = {"discussion_title": "L'ennesima difficoltà."}  # Sostituisci con i tuoi criteri di filtro
documenti = collezione.find(query)  # Ottieni i documenti corrispondenti

# Converti i documenti in una lista di oggetti Python
risultati = list(documenti)

In [26]:
risultati

[{'_id': ObjectId('6798ecbd85d5cfef9e3f2961'),
  'section_title': 'Presentiamoci',
  'section_link': 'https://quelledialfpma.forumfree.it//?f=64662418',
  'discussion_title': "L'ennesima difficoltà.",
  'discussion_link': 'https://quelledialfpma.forumfree.it//?t=80522994',
  'discussion_author': 'Ember',
  'author': 'Ember',
  'date': '26/12/2024',
  'time': '17:55',
  'message': "Buonasera a tutte, finalmente trovo il coraggio di scrivere su questo gruppo nella speranza di trovare un po' di sostegno e conforto. Due mesi fa io e mio marito abbiamo scoperto di non poter concepire naturalmente. La prima reazione è stata di rabbia furiosa e di sconforto ed anche adesso, ogni volta che ci penso, ho la tentazione di rinchiudermi in una stanza buia e piangere. Il percorso di PMA mi terrorizza ma soprattutto mi devasta l'idea di dover affrontare ancora una difficoltà di vita dopo averne superate tante. Un bimbo sarebbe dovuto arrivare naturalmente, senza pensieri, una circostanza felice; e in

In [24]:
messaggi_concatenati = ".".join(d["message"] for d in risultati if "message" in d)

In [25]:
messaggi_concatenati

'Buonasera a tutte, finalmente trovo il coraggio di scrivere su questo gruppo nella speranza di trovare un po\' di sostegno e conforto. Due mesi fa io e mio marito abbiamo scoperto di non poter concepire naturalmente. La prima reazione è stata di rabbia furiosa e di sconforto ed anche adesso, ogni volta che ci penso, ho la tentazione di rinchiudermi in una stanza buia e piangere. Il percorso di PMA mi terrorizza ma soprattutto mi devasta l\'idea di dover affrontare ancora una difficoltà di vita dopo averne superate tante. Un bimbo sarebbe dovuto arrivare naturalmente, senza pensieri, una circostanza felice; e invece ora ho incubi su medici, iniezioni e dolori. Non riesco a superare lo shock e nessuno, a partire da mio marito, capisce cosa provo; nessuno si rende conto del fatto che l\'intero peso fisico del percorso di PMA dovrò sopportarlo io. Poterne parlare qui è un sollievo, vi ringrazio.Ciao Ember, benvenuta :purple_heart: mi riconosco molto in quello che dici, e penso che tutte n